## Mobile Games A/B Testing with Cookie Cats
<p><a href="https://www.facebook.com/cookiecatsgame">Cookie Cats</a> is a hugely popular mobile puzzle game developed by <a href="http://tactile.dk">Tactile Entertainment</a>. It's a classic "connect three"-style puzzle game where the player must connect tiles of the same color to clear the board and win the level.
<p>As players progress through the levels of the game, they will occasionally encounter gates that force them to wait a non-trivial amount of time or make an in-app purchase to progress. In addition to driving in-app purchases, these gates serve the important purpose of giving players an enforced break from playing the game, hopefully resulting in that the player's enjoyment of the game being increased and prolonged.</p>
<p><img src="https://s3.amazonaws.com/assets.datacamp.com/production/project_184/img/cc_gates.png" alt=""></p>
<p>Initially the first gate was placed at level 30. This project is going to analyze an AB-test where the first gate in Cookie Cats was moved from level 30 to level 40.</p>

In [1]:
import pandas as pd

df = pd.read_csv('cookie_cats.csv')

print(df.head())
print('\n')
print('Rows of dataframe : ' + str(len(df)) + ', with ' + str(len(df.userid.unique())) + ' different players')

   userid  version  sum_gamerounds  retention_1  retention_7
0     116  gate_30               3        False        False
1     337  gate_30              38         True        False
2     377  gate_40             165         True        False
3     483  gate_40               1        False        False
4     488  gate_40             179         True         True


Rows of dataframe : 90189, with 90189 different players


<p>The data we have is from 90,189 players that installed the game while the AB-test was running. The variables are:</p>
<ul>
<li><code>userid</code> - a unique number that identifies each player.</li>
<li><code>version</code> - whether the player was put in the control group (<code>gate_30</code> - a gate at level 30) or the group with the moved gate (<code>gate_40</code> - a gate at level 40).</li>
<li><code>sum_gamerounds</code> - the number of game rounds played by the player during the first 14 days after install.</li>
<li><code>retention_1</code> - did the player come back and play <strong>1 day</strong> after installing?</li>
<li><code>retention_7</code> - did the player come back and play <strong>7 days</strong> after installing?</li>
</ul>
<p>When a player installed the game, he or she was randomly assigned to either <code>gate_30</code> or <code>gate_40</code>.

In [2]:
print('Number of players in each AB groups :')
ab_group = df[['version', 'userid']].groupby('version', as_index=False).userid.count()
ab_group.columns = ['version', 'number of players']
print(ab_group)

Number of players in each AB groups :
   version  number of players
0  gate_30              44700
1  gate_40              45489


In [3]:
import matplotlib.pyplot as plt
 
plot_df = df.groupby('sum_gamerounds').userid.count()

ax = plot_df.head(100).plot()
ax.set_xlabel("Number of game rounds")
ax.set_ylabel("Number of players")
plt.axis([-5, 105, -100, 5800])
plt.title('Distribution of players that played 0 to 100 game rounds during their first week playing the game')
print(plt.plot())

[]


In [4]:
import numpy as np
# AB Testing using Hypothesis Test Regarding Two Population Proportions
def z_two_proportion(x1, x2, n1, n2) :
    "Calculate test statistic z value for Hypothesis Test Regarding Two Population Proportions"
    p = (x1+x2)/(n1+n2)
    p1 = x1/n1
    p2 = x2/n2
    z = (p1-p2)/(np.sqrt(p*(1-p)*((1/n1)+(1/n2))))
    return round(z, 3)

#### 1-day retention analysis
<p>In the plot above we can see that some players install the game but then never play it (0 game rounds), some players just play a couple of game rounds in their first week, and some get really hooked!</p>
<p>What we want is for players to like the game and to get hooked. A common metric in the video gaming industry for how fun and engaging a game is <em>1-day retention</em>: The percentage of players that comes back and plays the game <em>one day</em> after they have installed it.  The higher 1-day retention is, the easier it is to retain players and build a large player base. </p>

In [5]:
retention_1day = round((df.retention_1.mean() * 100), 2)
print('Percentage of players that came back the day after they installed :', retention_1day, '%')

Percentage of players that came back the day after they installed : 44.52 %


In [6]:
ab_retention1 = df[['version', 'retention_1']].groupby('version', as_index=False).retention_1.mean()
ab_retention1.retention_1 = round((ab_retention1.retention_1 * 100), 2)
ab_retention1.columns = ['version', 'percentage of players']
print('1-day retention for each AB groups :')
print(ab_retention1)

1-day retention for each AB groups :
   version  percentage of players
0  gate_30                  44.82
1  gate_40                  44.23


<p>It appears that there was a slight decrease in 1-day retention when the gate was moved to level 40 (44.23%) compared to the control when it was at level 30 (44.82%). It's a small change, but even small changes in retention can have a large impact.</p>
<p>Perform a hypothesis test on the difference in the two population proportions using a level of significance α = 0.05 (i.e., 5%).</p>
<p>p1 = proportion of 1-day retention from gate_30 version</p>
<p>p2 = proportion of 1-day retention from gate_40 version</p>
<p>H0 : p1 <= p2</p>
<p>H1 : p1 > p2</p>

In [7]:
# Hypothesis test for 1-day retention analysis
x_30_1 = df[(df['version']=='gate_30')&(df['retention_1'])]['userid'].count()
x_40_1 = df[(df['version']=='gate_40')&(df['retention_1'])]['userid'].count()
n_30 = df[df['version']=='gate_30']['userid'].count()
n_40 = df[df['version']=='gate_40']['userid'].count()
z_retention_1 = z_two_proportion(x_30_1, x_40_1, n_30, n_40)
print('test statistic z :', z_retention_1)

test statistic z : 1.784


<p>With level of significance α = 0.05 (i.e., 5%) and z(0.05) = 1.645, so the test statistic z = 1.784 > z(0.05) = 1645.</p>
<p>The conclusion of the hypothesis test is : <strong>Reject the Null Hypothesis.</strong> There is sufficient evidence to say that <strong>1-day retention of gate_30 version is bigger than gate_40 version</strong> with level of significance 5%.</p>

#### 7-day retention analysis
<p>The hypothesis test tells us that there is sufficient evidence to say that 1-day retention is better when the gate is at level 30. However, since players have only been playing the game for one day, it is likely that most players haven't reached level 30 yet. That is, many players won't have been affected by the gate, even if it's as early as level 30. </p>
<p>But after having played for a week, more players should have reached level 40, and therefore it makes sense to also look at 7-day retention. That is: What percentage of the people that installed the game also showed up a week later to play the game again.</p>

In [8]:
ab_retention7 = df[['version', 'retention_7']].groupby('version', as_index=False).retention_7.mean()
ab_retention7.retention_7 = round((ab_retention7.retention_7 * 100), 2)
ab_retention7.columns = ['version', 'percentage of players']
print('7-day retention for each AB groups :')
print(ab_retention7)

7-day retention for each AB groups :
   version  percentage of players
0  gate_30                  19.02
1  gate_40                  18.20


<p>Like with 1-day retention, we see that 7-day retention is slightly lower (18.20%) when the gate is at level 40 than when the gate is at level 30 (19.02%). This difference is also larger than for 1-day retention, presumably because more players have had time to hit the first gate. We also see that the <em>overall</em> 7-day retention is lower than the <em>overall</em> 1-day retention; fewer people play a game a week after installing than a day after installing.</p>
<p>But as before, let's perform a hypothesis test on the difference in the two population proportions using a level of significance α = 0.05 (i.e., 5%).</p>
<p>p1 = proportion of 7-day retention from gate_30 version</p>
<p>p2 = proportion of 7-day retention from gate_40 version</p>
<p>H0 : p1 <= p2</p>
<p>H1 : p1 > p2</p>

In [9]:
# Hypothesis test for 1-day retention analysis
x_30_7 = df[(df['version']=='gate_30')&(df['retention_7'])]['userid'].count()
x_40_7 = df[(df['version']=='gate_40')&(df['retention_7'])]['userid'].count()
n_30 = df[df['version']=='gate_30']['userid'].count()
n_40 = df[df['version']=='gate_40']['userid'].count()
z_retention_7 = z_two_proportion(x_30_7, x_40_7, n_30, n_40)
print('test statistic z :', z_retention_7)

test statistic z : 3.164


<p>With level of significance α = 0.05 (i.e., 5%) and z(0.05) = 1.645, so the test statistic z = 3.164 > z(0.05) = 1645.</p>
<p>The conclusion of the hypothesis test is : <strong>Reject the Null Hypothesis.</strong> There is sufficient evidence to say that <strong>7-day retention of gate_30 version is bigger than gate_40 version</strong> with level of significance 5%.</p>

### The conclusion
<p>The hypothesis test tells us that there is sufficient evidence that 7-day retention is higher when the gate is at level 30 than when it is at level 40. The conclusion is: If we want to keep retention high — both 1-day and 7-day retention — we should <strong>not</strong> move the gate from level 30 to level 40. There are, of course, other metrics we could look at, like the number of game rounds played or how much in-game purchases are made by the two AB-groups. But retention <em>is</em> one of the most important metrics. If we don't retain our player base, it doesn't matter how much money they spend in-game.</p>
<p>So, why is retention higher when the gate is positioned earlier? One could expect the opposite: The later the obstacle, the longer people are going to engage with the game. But this is not what the data tells us. The theory of <em>hedonic adaptation</em> can give one explanation for this. In short, hedonic adaptation is the tendency for people to get less and less enjoyment out of a fun activity over time if that activity is undertaken continuously. By forcing players to take a break when they reach a gate, their enjoyment of the game is prolonged. But when the gate is moved to level 40, fewer players make it far enough, and they are more likely to quit the game because they simply got bored of it. </p>